# Capstone - Creating your own custom dataset

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import random


In [2]:
model_name = "microsoft/phi-2"
phi2 = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype = torch.float16
        ).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
phi2.config.pad_token_id = tokenizer.eos_token_id


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
#Select a seed word - "Data"
seed_word = "machine learning"

In [4]:
def getTextData(word):
    # Tokenize input prompt
    input_ids = tokenizer.encode(word, return_tensors="pt").to("cuda")
    attention_mask = (input_ids != tokenizer.pad_token_id).float()
    # Generate text
    output = phi2.generate(input_ids, max_length=50, attention_mask=attention_mask)
    # Decode generated output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)    
    return generated_text

In [5]:
from datasets import Dataset

# Initialize an empty list to store the text data
text_data_list = []

# Your loop that runs 50,000 times to gather text data
for i in range(10000):
    if(i == 0):
        text_data = getTextData(seed_word)
    else:
        words = re.findall(r'\b\w+\b', text_data)
        text_data = getTextData(random.choice(words))
        
    text_data_list.append({"text": text_data})


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [6]:
len(text_data_list)

10000

In [7]:
# Create a dictionary with the collected text data
data_dict = {"text": [item["text"] for item in text_data_list]}

# Create a Dataset using the collected data
my_dataset = Dataset.from_dict(data_dict)

# Create a dictionary with the collected text data
data_dict = {"text": [item["text"] for item in text_data_list]}

# Create a Dataset using the collected data
my_dataset = Dataset.from_dict(data_dict)

# Save the dataset using torch.save
my_dataset.save_to_disk("/kaggle/working/custom_dataset")

In [8]:
loaded_dataset = Dataset.load_from_disk("/kaggle/working/custom_dataset")

In [9]:
loaded_dataset[0]

{'text': 'machine learning, and artificial intelligence.\n\nExercise 2: What is the difference between a computer and a tablet?\nAnswer: A computer is a larger device that typically has a keyboard and a mouse, while a tablet is a smaller device that'}